In [1]:
import pandas as pd
import numpy as np 
import main 
import pathlib

In [2]:
################    1. read P measure parameters and compute CDS^P    ##################


# which countries to include
country_names = main.TOTAL_COUNTRY_NAMES

# DATA_V = 'AEJ_data'
DATA_V = 'EXT_data'

# DATE_FOLDER = "2021_8_20"
DATE_FOLDER = "2021_8_23"


In [3]:
df = pd.read_csv(f"outputs/MLE/{DATA_V}/{DATE_FOLDER}/estimation_table.csv", index_col=0)
p_param = df[['ALP', 'BET', 'SIG', 'ALPH', 'BETH', 'SIG1', 'SIG2']].astype(float)

In [4]:
for i, country in enumerate(p_param.index):

    # model parameters
    CDS1Y, CDS3Y, CDS5Y, PFILE, _ = main.get_input(ICAR_1=i, DATA_V=DATA_V)
    PARM = p_param.loc[country].values

    # CDS output path
    CDS_path = f"outputs/CDS/{DATA_V}/{DATE_FOLDER}/"
    pathlib.Path(CDS_path).mkdir(parents=True, exist_ok=True)

    # compute CDS spreads
    main.FUNC_NLLK(PARM=PARM, CDS1Y=CDS1Y, CDS3Y=CDS3Y, CDS5Y=CDS5Y, PFILE=PFILE, country_name=country, 
                    XB_FLAG=False, LN_FLAG=False, logging=False, print_flag=False,
                    CDSLog_flag=True, CDSLog_path=CDS_path) # important
    # break

Brazil 2001-11-30 - 2021-7-1 # obs: 236
[ 0.494 -0.125  0.689 -2.006  0.453  0.026  0.009],-1369.9023529815272, 0.017506022636673956
Bulgaria 2003-1-31 - 2014-3-31 # obs: 134
[-4.330e-01  1.010e-01  1.084e+00 -1.672e+00  3.020e-01  2.000e-03
  1.000e-03],-1323.535941062298, 0.0013273019212919626
Chile 2003-1-31 - 2021-7-1 # obs: 222
[-4.390e-01  5.800e-02  1.407e+00 -2.820e+00  4.280e-01  1.000e-03
  1.000e-03],-2510.0128848584754, 0.0005390482374397436
China 2003-1-31 - 2010-1-29 # obs: 84


In [ ]:
for i, country in enumerate(p_param.index):

    # risk-premium file output path
    RP_path = f"outputs/risk_premium/{DATA_V}/{DATE_FOLDER}/"
    pathlib.Path(RP_path).mkdir(parents=True, exist_ok=True)

    # read CDS
    CDS = pd.read_csv(f"{CDS_path}/{country}_CDS.csv") * 10000

    # compute and save RP
    tenor = 5
    CDS[f'RP'] = CDS[f"CDS{tenor}Y"] - CDS[f"CDS{tenor}XP"]
    CDS[f'RP_frac'] = ((CDS[f"CDS{tenor}Y"] - CDS[f"CDS{tenor}XP"]) / CDS[f"CDS{tenor}Y"]) * 100 # unit in %
    CDS[['RP', 'RP_frac']].to_csv(f"{RP_path}/{country}_RP.csv")

    # break